In [1]:
#!cat /proc/cpuinfo
#!cat /proc/meminfo

In [2]:
!pip install pyshark
!apt-get install tshark

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
/bin/sh: apt-get: command not found


In [3]:
import pyshark
import pandas as pd


**You can upload any file with google drive id (follow line 1~15)**

**-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------**

In [4]:
http = pyshark.FileCapture('http.pcap')
http.load_packets() 
dns = pyshark.FileCapture('dns.pcap', keep_packets=True, display_filter='dns')

In [5]:
http[0].show()

Layer ETH:
	Destination: 02:42:ac:11:00:02
	Address: 02:42:ac:11:00:02
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 02:42:63:79:49:dd
	Type: IPv4 (0x0800)
	Address: 02:42:63:79:49:dd
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
Layer IP:
	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 44
	Identification: 0x2dda (11738)
	Flags: 0x00
	0... .... = Reserved bit: Not set
	.0.. .... = Don't fragment: Not set
	..0. .... = More fragments: Not set
	Fragment offset: 0
	Time to live: 63
	Protocol: TCP (6)
	

In [6]:
http[1].show()

Layer ETH:
	Destination: 02:42:63:79:49:dd
	Address: 02:42:63:79:49:dd
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
	Source: 02:42:ac:11:00:02
	Type: IPv4 (0x0800)
	Address: 02:42:ac:11:00:02
	.... ..1. .... .... .... .... = LG bit: Locally administered address (this is NOT the factory default)
	.... ...0 .... .... .... .... = IG bit: Individual address (unicast)
Layer IP:
	0100 .... = Version: 4
	.... 0101 = Header Length: 20 bytes (5)
	Differentiated Services Field: 0x00 (DSCP: CS0, ECN: Not-ECT)
	0000 00.. = Differentiated Services Codepoint: Default (0)
	.... ..00 = Explicit Congestion Notification: Not ECN-Capable Transport (0)
	Total Length: 44
	Identification: 0x0000 (0)
	Flags: 0x02 (Don't Fragment)
	0... .... = Reserved bit: Not set
	.1.. .... = Don't fragment: Set
	..0. .... = More fragments: Not set
	Fragment offset: 0
	Time to live: 64
	Protocol: 

In [7]:
len(http)

604

In [8]:
dir(http[2])

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_packet_string',
 'captured_length',
 'eth',
 'frame_info',
 'get_multiple_layers',
 'highest_layer',
 'interface_captured',
 'ip',
 'layers',
 'length',
 'number',
 'pretty_print',
 'show',
 'sniff_time',
 'sniff_timestamp',
 'tcp',
 'transport_layer']

In [9]:
df = pd.read_json('osqueryd.results.log',  lines=True, orient='records')

In [10]:
df['columns'][1]

{'action': 'CREATED',
 'atime': '1517877228',
 'category': 'etc',
 'ctime': '1517877228',
 'gid': '0',
 'hashed': '1',
 'inode': '159766',
 'md5': '0d4f52b9dacac07cf2316d1686cc707c',
 'mode': '0644',
 'mtime': '1517877228',
 'sha1': 'cffc9b8ead76006419975c0d22ba49d5fdf001a0',
 'sha256': '43d11069f7682e63289f38d184e4193f7486502f099250ceb4af7ac4bffb8209',
 'size': '4096',
 'target_path': '/etc/osquery/.osquery.conf.swp',
 'time': '1517877228',
 'transaction_id': '0',
 'uid': '0'}

In [11]:
from pandas.io.json import json_normalize

In [12]:
test = [1,2,3]

In [13]:
df.columns.rename

<bound method Index.rename of Index(['action', 'calendarTime', 'columns', 'counter', 'decorations', 'epoch',
       'hostIdentifier', 'name', 'unixTime'],
      dtype='object')>

In [14]:
df_norm = (pd.concat([df, json_normalize(list(df['columns'])).rename(columns=lambda x: ('columns_' + x))], axis=1)).drop(columns=['columns'])
df_norm = (pd.concat([df_norm, json_normalize(list(df_norm['decorations'])).rename(columns=lambda x: ('decorations_' + x))], axis=1)).drop(columns=['decorations'])

In [15]:
df_norm.to_csv('log.csv')

In [16]:
try:
    i = 0
    while True:
        dns[i]
        i += 1
except KeyError:
    pass


# Task 1

## 1.

### 1.a Report the UNIX timestamp of the first attempted scan on the vulnerable server

In [17]:
for i in range(0, len(http)):
  if ('http' in dir(http[i])) and 'request_uri' in dir(http[i].http) and (http[i].http.request_uri == '/showcase.action'):
    print(http[i].sniff_timestamp)
    break

1521394903.610774000


#### *!!!!!!!!!!!!!!!!!!!!Might be http not GET*

### 1.b Identify the IP address of the vulnerable server

In [18]:
for i in range(0, len(http)):
  if ('http' in dir(http[i])) and 'request_uri' in dir(http[i].http) and (http[i].http.request_uri == '/showcase.action'):
    print(http[i].ip.dst)
    break

172.17.0.2


### 1.c Identify the port of the vulnerable server

In [19]:
for i in range(0, len(http)):
  if ('http' in dir(http[i])) and 'request_uri' in dir(http[i].http) and (http[i].http.request_uri == '/showcase.action'):
    print(http[i].tcp.dstport)
    break

8080


## 2.

### 2.a Extract a list of the Content-Type headers sent to the vulnerable server from the provided HTTP packet capture. For each Content-Type header, provide its length as well.

In [20]:
for i in range(0, len(http)):
    if ('http' in dir(http[i])) and 'request_uri' in dir(http[i].http) and (http[i].http.request_uri == '/showcase.action') and (http[i].http.get('content-type') != None):
        print('index: ' + str(i))
        print(http[i].http.get('content-type'))
        print("Content-type length: " + str(len(http[i].http.get('content-type'))))

index: 413
.multipart/form-data~${#context["com.opensymphony.xwork2.dispatcher.HttpServletResponse"].addHeader("LOLOLOLOLOLPAYLOADWORKEDLOLOLOLOL",1330+7)}
Content-type length: 144
index: 434
.multipart/form-data~${#context["com.opensymphony.xwork2.dispatcher.HttpServletResponse"].addHeader("LOLOLOLOLOLPAYLOADWORKEDLOLOLOLOL",1330+7)}
Content-type length: 144
index: 449
.multipart/form-data~${#context["com.opensymphony.xwork2.dispatcher.HttpServletResponse"].addHeader("LOLOLOLOLOLPAYLOADWORKEDLOLOLOLOL",1330+7)}
Content-type length: 144
index: 524
%{(#_='multipart/form-data').(#dm=@ognl.OgnlContext@DEFAULT_MEMBER_ACCESS).(#_memberAccess?(#_memberAccess=#dm):((#container=#context['com.opensymphony.xwork2.ActionContext.container']).(#ognlUtil=#container.getInstance(@com.opensymphony.xwork2.ognl.OgnlUtil@class)).(#ognlUtil.getExcludedPackageNames().clear()).(#ognlUtil.getExcludedClasses().clear()).(#context.setMemberAccess(#dm)))).(#cmd='ls').(#iswin=(@java.lang.System@getProperty('os.nam

### 2.b Fill in the blanks in the table below.

![alt text](https://i.imgur.com/ofPoyfw.png)

## 3.

Provide a list of kernel modules added or removed from the system


In [21]:
df_norm.columns

Index(['action', 'calendarTime', 'counter', 'epoch', 'hostIdentifier', 'name',
       'unixTime', 'columns_action', 'columns_atime', 'columns_category',
       'columns_ctime', 'columns_gid', 'columns_hashed', 'columns_inode',
       'columns_md5', 'columns_mode', 'columns_mtime', 'columns_name',
       'columns_sha1', 'columns_sha256', 'columns_size', 'columns_target_path',
       'columns_time', 'columns_transaction_id', 'columns_uid',
       'decorations_host_uuid', 'decorations_username'],
      dtype='object')

**!!!!!!!!!!!!!!!!!!!!!!NOT SURE**

In [22]:
df_norm['action'].unique()

array(['added', 'removed'], dtype=object)

In [23]:
df_norm[(df_norm['action'] == 'added') | (df_norm['action'] == 'removed')]

,action,calendarTime,counter,epoch,hostIdentifier,name,unixTime,columns_action,columns_atime,columns_category,...,columns_name,columns_sha1,columns_sha256,columns_size,columns_target_path,columns_time,columns_transaction_id,columns_uid,decorations_host_uuid,decorations_username
0,added,Tue Feb 6 00:33:05 2018 UTC,38463,0,ubuntu,system_info,1517877185,NaN,NaN,NaN,...,rk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
1,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,CREATED,1517877228,etc,...,NaN,cffc9b8ead76006419975c0d22ba49d5fdf001a0,43d11069f7682e63289f38d184e4193f7486502f099250...,4096,/etc/osquery/.osquery.conf.swp,1517877228,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
2,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,CREATED,,etc,...,NaN,,,,/etc/osquery/.osquery.conf.swx,1517877228,0,,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
3,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,UPDATED,,etc,...,NaN,,,,/etc/osquery/.osquery.conf.swx,1517877228,0,,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
4,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,DELETED,,etc,...,NaN,,,,/etc/osquery/.osquery.conf.swx,1517877228,0,,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
5,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,UPDATED,1517877228,etc,...,NaN,cffc9b8ead76006419975c0d22ba49d5fdf001a0,43d11069f7682e63289f38d184e4193f7486502f099250...,4096,/etc/osquery/.osquery.conf.swp,1517877228,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
6,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,DELETED,1517877228,etc,...,NaN,,,4096,/etc/osquery/.osquery.conf.swp,1517877228,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
7,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,CREATED,1517877228,etc,...,NaN,cffc9b8ead76006419975c0d22ba49d5fdf001a0,43d11069f7682e63289f38d184e4193f7486502f099250...,4096,/etc/osquery/.osquery.conf.swp,1517877228,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
8,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,UPDATED,1517877228,etc,...,NaN,cffc9b8ead76006419975c0d22ba49d5fdf001a0,43d11069f7682e63289f38d184e4193f7486502f099250...,4096,/etc/osquery/.osquery.conf.swp,1517877228,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
9,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,ATTRIBUTES_MODIFIED,1517877228,etc,...,NaN,,,4096,/etc/osquery/.osquery.conf.swp,1517877228,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root


In [24]:
df_norm.head()

,action,calendarTime,counter,epoch,hostIdentifier,name,unixTime,columns_action,columns_atime,columns_category,...,columns_name,columns_sha1,columns_sha256,columns_size,columns_target_path,columns_time,columns_transaction_id,columns_uid,decorations_host_uuid,decorations_username
0,added,Tue Feb 6 00:33:05 2018 UTC,38463,0,ubuntu,system_info,1517877185,NaN,NaN,NaN,...,rk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
1,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,CREATED,1517877228,etc,...,NaN,cffc9b8ead76006419975c0d22ba49d5fdf001a0,43d11069f7682e63289f38d184e4193f7486502f099250...,4096,/etc/osquery/.osquery.conf.swp,1517877228,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
2,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,CREATED,,etc,...,NaN,,,,/etc/osquery/.osquery.conf.swx,1517877228,0,,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
3,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,UPDATED,,etc,...,NaN,,,,/etc/osquery/.osquery.conf.swx,1517877228,0,,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root
4,added,Tue Feb 6 00:33:49 2018 UTC,0,0,ubuntu,pack_fim_file_events,1517877229,DELETED,,etc,...,NaN,,,,/etc/osquery/.osquery.conf.swx,1517877228,0,,D5882FBF-1D65-4A30-B216-77F664B7D3B0,root


In [25]:
df_norm['name'].unique()

array(['system_info', 'pack_fim_file_events', 'kernel_module'],
      dtype=object)

In [26]:
df_norm[df_norm['name']=='kernel_module']

,action,calendarTime,counter,epoch,hostIdentifier,name,unixTime,columns_action,columns_atime,columns_category,...,columns_name,columns_sha1,columns_sha256,columns_size,columns_target_path,columns_time,columns_transaction_id,columns_uid,decorations_host_uuid,decorations_username
42,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,rk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
43,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,ipt_MASQUERADE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
44,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,nf_nat_masquerade_ipv4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
45,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,nf_conntrack_netlink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
46,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,nfnetlink,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
47,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,xfrm_user,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
48,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,xfrm_algo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
49,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,iptable_nat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
50,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,nf_conntrack_ipv4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,
51,added,Tue Feb 6 00:34:09 2018 UTC,0,0,ubuntu,kernel_module,1517877249,NaN,NaN,NaN,...,nf_defrag_ipv4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,D5882FBF-1D65-4A30-B216-77F664B7D3B0,


In [27]:
df_norm['columns_name'].unique()

array(['rk', nan, 'ipt_MASQUERADE', 'nf_nat_masquerade_ipv4',
       'nf_conntrack_netlink', 'nfnetlink', 'xfrm_user', 'xfrm_algo',
       'iptable_nat', 'nf_conntrack_ipv4', 'nf_defrag_ipv4',
       'nf_nat_ipv4', 'xt_addrtype', 'iptable_filter', 'ip_tables',
       'xt_conntrack', 'x_tables', 'nf_nat', 'nf_conntrack',
       'br_netfilter', 'bridge', 'stp', 'llc', 'overlay', 'ppdev',
       'intel_powerclamp', 'crct10dif_pclmul', 'crc32_pclmul',
       'ghash_clmulni_intel', 'aesni_intel', 'aes_x86_64', 'lrw',
       'vboxvideo', 'gf128mul', 'glue_helper', 'ablk_helper', 'cryptd',
       'ttm', 'drm_kms_helper', 'snd_intel8x0', 'snd_ac97_codec',
       'ac97_bus', 'input_leds', 'joydev', 'serio_raw', 'snd_pcm', 'drm',
       'fb_sys_fops', 'snd_timer', 'syscopyarea', 'sysfillrect',
       'i2c_piix4', 'snd', 'sysimgblt', 'soundcore', 'vboxguest',
       '8250_fintek', 'parport_pc', 'parport', 'mac_hid', 'autofs4',
       'hid_generic', 'usbhid', 'hid', 'psmouse', 'ahci', 'libahci',
 

The kernel module that the attacker obtained in Task 1.2 has been loaded into the vulnerable server because "rk" is in the names.

In [28]:
df_norm[df_norm['columns_target_path']=='/root/.ssh/id_rsa']

,action,calendarTime,counter,epoch,hostIdentifier,name,unixTime,columns_action,columns_atime,columns_category,...,columns_name,columns_sha1,columns_sha256,columns_size,columns_target_path,columns_time,columns_transaction_id,columns_uid,decorations_host_uuid,decorations_username
2352,added,Sun Mar 18 18:19:47 2018 UTC,0,0,ubuntu,pack_fim_file_events,1521397187,CREATED,1521397187,root,...,NaN,e54ada49974238299758017ad64a0a87e59ff47a,851056543e7c262e3d4356e499ea25cb77aa73f4f11286...,1679,/root/.ssh/id_rsa,1521397187,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,reboot
2353,added,Sun Mar 18 18:19:47 2018 UTC,0,0,ubuntu,pack_fim_file_events,1521397187,UPDATED,1521397187,root,...,NaN,e54ada49974238299758017ad64a0a87e59ff47a,851056543e7c262e3d4356e499ea25cb77aa73f4f11286...,1679,/root/.ssh/id_rsa,1521397187,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,reboot
2354,added,Sun Mar 18 18:19:47 2018 UTC,0,0,ubuntu,pack_fim_file_events,1521397187,UPDATED,1521397187,root,...,NaN,e54ada49974238299758017ad64a0a87e59ff47a,851056543e7c262e3d4356e499ea25cb77aa73f4f11286...,1679,/root/.ssh/id_rsa,1521397187,0,0,D5882FBF-1D65-4A30-B216-77F664B7D3B0,reboot


# 5.

In [29]:
ETH = 0
IP = 1
TCP = 2
dest_ip = http[2][IP].dst
dest_port = http[2][TCP].dstport
print(dest_ip, ':',dest_port)

172.17.0.2 : 8080


In [30]:
df_data = []
for pkt in http:
    if len(pkt.layers) < 3:
        continue
    if pkt[2].layer_name != 'tcp':
        continue
    
    d = {
        'timestamp': pkt.sniff_time.timestamp(),
        'dest_ip': pkt[IP].dst,
        'dest_port': pkt[TCP].dstport
    }
    df_data.append(d)

In [31]:
df_http = pd.DataFrame(df_data)
df_http.to_csv("http_df.csv", sep='\t')
df_http.head()

,dest_ip,dest_port,timestamp
0,172.17.0.2,8080,1.521395e+09
1,10.0.2.2,55952,1.521395e+09
2,172.17.0.2,8080,1.521395e+09
3,172.17.0.2,8080,1.521395e+09
4,10.0.2.2,55952,1.521395e+09


In [32]:
dest_ip = dns[2][IP].dst
dest_port = dns[2][TCP].dstport
print(dest_ip, ':',dest_port)
df_data = []
for pkt in dns:
    if len(pkt.layers) < 3:
        continue
    if pkt[2].layer_name != 'udp':
        continue
    
    d = {
        'timestamp': pkt.sniff_time.timestamp(),
        'dest_ip': pkt[IP].dst,
        'dest_port': pkt[TCP].dstport,
        'server':pkt.dns.flags_response,
        'length': pkt.dns.qry_name_len
    }
    df_data.append(d)
df_dns = pd.DataFrame(df_data)
df_dns.to_csv("http_df.csv", sep='\t')


162.212.156.148 : 53


In [43]:
df_dns['dest_ip'].unique()

array(['162.212.156.148', '10.0.2.15', '75.75.75.75'], dtype=object)

In [35]:
df_filter=df_dns[df_dns.server == '0']
df_filter['dest_ip'].unique()
df_att_ctr=df_dns[df_dns.dest_ip=='162.212.156.148']
df_att_ctr['length'].unique()

array(['17', '20', '41', '15', '50', '59', '65', '45', '61', '18', '119',
       '91', '86', '129', '253', '232', '176', '139', '97', '140', '162',
       '107', '154', '192', '222', '160', '168', '66', '149', '152',
       '150', '171', '131', '180', '132', '109', '159', '26', '190',
       '184', '199', '197', '70', '69', '73', '92', '87', '231', '95',
       '143', '118'], dtype=object)

In [36]:
df_legi=df_dns[df_dns.dest_ip=='75.75.75.75']
df_legi['length'].unique()

array(['10', '27', '9', '32', '30', '20', '14', '19', '21', '33'],
      dtype=object)

162.212.156.148 is the attacker controlled server, 75.75.75.75 is legitimate DNS server

In [37]:
import matplotlib.pyplot as plt
import numpy as np

list1 = df_att_ctr['length']
list2 = df_legi['length']


fig, ax = plt.subplots(1,2)
ax[0].hist(list1,  color = 'r')
ax[1].hist(list2,  color = 'g')
plt.show()